In [1]:
import requests
import math
import time
from tqdm import tqdm 

In [2]:
import json
data = None
with open("raw.json", encoding="utf-8") as fin:
    data = json.load(fin)


In [32]:
from bs4 import BeautifulSoup
import re
from bs4 import NavigableString, Comment

dictionary = {}
idx = 0
for ch in tqdm(data, total=len(data)):
    idx += 1
    if idx < 100:
        continue
        
    d = data[ch]
    soup = BeautifulSoup(f"<body>{d['raw']}</body>", 'html.parser')
    dom = soup.select_one("body > div > div.panel-title")
    word = dom.text.strip().replace("ผลการค้นหา", "").replace("\"","").strip()
    
    meanings = []
    for dom in soup.select("body > div > div.panel-body > div"):
        title = dom.select_one(".panel-title").text
        means = []
        
        extra = {}
        for child in dom.select_one(".panel-body"):
            if isinstance(child, NavigableString):
                means.append(child.replace(u'\xa0', u' ').strip())
            else:
                if child.name == "br": 
                    continue
                elif child.name == "i":
                    means.append(f"**{child.text.strip()}**")
                    continue
                elif child.name == "b":
                    means.append(f"__{child.text.strip()}__")
                    continue
                elif child.name == "u":
                    means.append(f"`{child.text.strip()}`")
                    continue
                elif child.name == "sub":
                    means.append(f"{child.text.strip()}")
                    continue
                elif child.name == "sup":
                    means.append(f"{child.text.strip()}")
                    continue
                elif child.name == "img":
#                     means.append(f"[{child.src}]")
                    continue
                elif child.name == "center":
                    img = child.img["src"]
                    if "images" in extra:
                        extra["images"].append(img)
                    else:
                        extra["images"] = [img]
                elif child.name !="div":
                    print(dom.select_one(".panel-body"))
                    print("--------------------")
                    print(child)
                    print(child.img["src"])
                    raise(Exception(f"Unknow tag"))
                    continue
                    
                    
                extra_type = ""
                extra_val = []
                for c in child:
                    if isinstance(c, NavigableString):
                        t = c.strip()
                        if t=="":
                            continue
                        elif extra_type!="":
                            print(child)
                            raise(Exception(f"Many types"))
                        elif t.startswith("ลูกคำ"):
                            extra_type = "children"
                        elif t.startswith("แม่คำ"):
                            extra_type = "parent"
                        elif t.startswith("ความหมายที่เกี่ยวข้อง"):
                            extra_type = "related"
                        else:
                            print(word)
                            raise(Exception(f"Unknow type: {t}"))
                    else:
                        extra_val.append(c.text.strip())
                extra[extra_type] = extra_val
        o = {
            "word": title,
            "means": means,
            "extra": extra
        }
        meanings.append(o)
    dictionary[word] = meanings
        
#     break

100%|███████████████████████████████████████████████████████████████████████████| 39184/39184 [02:19<00:00, 280.33it/s]


In [52]:
for k in dictionary:
    word = dictionary[k]
    for mn in word:
        mn["word"] = mn["word"].strip()

In [57]:
import json
with open("dictionary.jsonl", "w", encoding="utf-8") as fout:
    for k in dictionary:
        entry = dictionary[k]
        json.dump(entry, fout, ensure_ascii=False)
        fout.write('\n')


# Categorized words

In [64]:
cat_words = {
    "spoken": [],
    "local": [],
#     "grammar": [],
    "proverb": []
}
for k in dictionary:
    word = dictionary[k]
    for mn in word:
        mn["word"] = mn["word"].strip()
        mean = " ".join(mn["means"])
        if "(ปาก)" in mean:
            cat_words["spoken"].append(mn)
        
        if "(ถิ่น)" in mean:
            cat_words["local"].append(mn)
        
#         if "(ไว)" in mean:
#             cat_words["grammar"].append(mn)
        
        if "(สำ)" in mean:
            cat_words["proverb"].append(mn)
            
        
    

In [63]:
for cat in cat_words:
for w in cat_words["grammar"]:
    print(w)

{'word': 'กรรตุ, กรรตุ-', 'means': ['[กัด, กัดตุ-, กันตุ-]  (ไว) น. ผู้ทำ. (ป. กตฺตุ; ส. กรฺตฤ).'], 'extra': {'children': ['กรรตุการก', 'กรรตุวาจก', 'กรรตุสัญญา']}}
{'word': 'กรรตุ, กรรตุ-', 'means': ['[กัด, กัดตุ-, กันตุ-]  (ไว) น. ผู้ทำ. (ป. กตฺตุ; ส. กรฺตฤ).'], 'extra': {'children': ['กรรตุการก', 'กรรตุวาจก', 'กรรตุสัญญา']}}
{'word': 'กรรตุการก', 'means': ['[กัดตุ-, กันตุ-]  (ไว) น. ผู้ทำหรือผู้ใช้ให้ทำ เป็นส่วนสำคัญส่วนหนึ่งใน ๒ ส่วนของประโยคที่ไม่ต้องมีกรรมมารับ เช่น เด็กวิ่ง เด็ก เป็น กรรตุการก หรือเป็นส่วนสำคัญส่วนหนึ่งใน ๓ ส่วนของประโยคที่กริยาต้องมีกรรมรับ เช่น ตำรวจยิงผู้ร้าย ตำรวจ เป็น กรรตุการก. (ป., ส. การก ว่า ผู้ทำ).'], 'extra': {'parent': ['กรรตุ', 'กรรตุ-']}}
{'word': 'กรรตุวาจก', 'means': ['[กัดตุ-, กันตุ-]  (ไว) ก. กริยาที่บอกว่าประธานเป็นกรรตุการกคือผู้ทำหรือผู้ใช้ให้ทำ, กริยาของประโยคที่แสดงว่าประธานทำหน้าที่เป็นกรรตุการกคือเป็นผู้ทำหรือผู้ใช้ให้ทำ เช่น', '**ครู**', 'เขียนหนังสือ (ประธานเป็นผู้ทำ)', '**ครู**', 'ให้นักเรียนเขียนหนังสือ (ประธานเป็นผู้ใช้ให้ทำ). (ป., 